In [24]:
from src.service.HwpService import HwpService

service = HwpService()
filePath = '/Users/dobby/Desktop/Workspace/3월 계측보고서(신안산선2공구A-1)_복잡이.hwp'

xml = service.hwp2xml(filePath)
columnTag, textTag = service.findTag(xml, '일자별 발파 및 계측 현황')
xmlData = service.setTableData(columnTag, textTag)

unmatched field end


In [25]:
tableId = sorted(set([i['table-id'] for i in xmlData]))
xmlData = [[i for i in xmlData if i['table-id'] == id] for id in tableId]

xmlData

[[{'table-id': 72, 'row': 0, 'rowspan': 1, 'col': 0, 'colspan': 1, 'text': ''},
  {'table-id': 72,
   'row': 0,
   'rowspan': 1,
   'col': 1,
   'colspan': 1,
   'text': ''}],
 [{'table-id': 73,
   'row': 0,
   'rowspan': 2,
   'col': 0,
   'colspan': 1,
   'text': '일시'},
  {'table-id': 73,
   'row': 0,
   'rowspan': 2,
   'col': 1,
   'colspan': 1,
   'text': '발파횟수'},
  {'table-id': 73,
   'row': 0,
   'rowspan': 2,
   'col': 2,
   'colspan': 1,
   'text': '시간'},
  {'table-id': 73,
   'row': 0,
   'rowspan': 1,
   'col': 3,
   'colspan': 2,
   'text': '장약량(kg)'},
  {'table-id': 73,
   'row': 0,
   'rowspan': 1,
   'col': 5,
   'colspan': 1,
   'text': '발파위치'},
  {'table-id': 73,
   'row': 0,
   'rowspan': 1,
   'col': 6,
   'colspan': 4,
   'text': '발파진동 및 소음 측정치(max값)'},
  {'table-id': 73,
   'row': 1,
   'rowspan': 1,
   'col': 3,
   'colspan': 1,
   'text': '지발당 장약량'},
  {'table-id': 73,
   'row': 1,
   'rowspan': 1,
   'col': 4,
   'colspan': 1,
   'text': '총장약량'},
  {'table-id': 

In [26]:
targetText = [ '일자', '시간', '발파위치', '진동속도', '진동레벨', '소음', '측정위치' ]

filteredData = [
    sublist for sublist in xmlData
    if any( entry['row'] in [0, 1] and any(keyword in entry['text'] for keyword in targetText) for entry in sublist )
]

filteredData

[[{'table-id': 73,
   'row': 0,
   'rowspan': 2,
   'col': 0,
   'colspan': 1,
   'text': '일시'},
  {'table-id': 73,
   'row': 0,
   'rowspan': 2,
   'col': 1,
   'colspan': 1,
   'text': '발파횟수'},
  {'table-id': 73,
   'row': 0,
   'rowspan': 2,
   'col': 2,
   'colspan': 1,
   'text': '시간'},
  {'table-id': 73,
   'row': 0,
   'rowspan': 1,
   'col': 3,
   'colspan': 2,
   'text': '장약량(kg)'},
  {'table-id': 73,
   'row': 0,
   'rowspan': 1,
   'col': 5,
   'colspan': 1,
   'text': '발파위치'},
  {'table-id': 73,
   'row': 0,
   'rowspan': 1,
   'col': 6,
   'colspan': 4,
   'text': '발파진동 및 소음 측정치(max값)'},
  {'table-id': 73,
   'row': 1,
   'rowspan': 1,
   'col': 3,
   'colspan': 1,
   'text': '지발당 장약량'},
  {'table-id': 73,
   'row': 1,
   'rowspan': 1,
   'col': 4,
   'colspan': 1,
   'text': '총장약량'},
  {'table-id': 73,
   'row': 1,
   'rowspan': 1,
   'col': 5,
   'colspan': 1,
   'text': 'STA'},
  {'table-id': 73,
   'row': 1,
   'rowspan': 1,
   'col': 6,
   'colspan': 1,
   'text': '발파

In [37]:
import copy, ast
from collections import defaultdict, Counter
from itertools import cycle, chain
from src.dto.CustomDefaultDict import CustomDefaultDict
        
def expandData(data: dict):
    expandData = []

    for i in range(data['colspan']):
        tempData = copy.deepcopy(data)
        tempData['colspan'] = 1
        tempData['col'] += i

        expandData.append(tempData)

    return expandData

def createRange(dataList: list):
    mergeDataList = [mergeData for mergeData in dataList if mergeData['rowspan'] > 1]
    
    mergeRanges = CustomDefaultDict()
    for data in mergeDataList:
        row = data['row']
        if data['col'] == 0:
            mergeRanges.append(f'{data}', list(range(row + 1, row + data['rowspan'] - 1)))
        else:
            mergeRanges.append(f'{data}', list(range(row + 1, row + data['rowspan'])))
    
    return dict(mergeRanges)
    

def test2(xmlDataList: list[dict]):
    delColumns = ['장약량(kg)', '발파위치', '발파진동 및 소음 측정치(max값)']
    columns = [column for column in xmlDataList if column['row'] in [0, 1] and column['text'] not in delColumns]
    
    print(Counter([xmldata['row'] for xmldata in xmlDataList if xmldata['row'] not in [0, 1]]))
    # rows = Counter(xmldata for xmldata in xmlDataList if xmldata['row'] not in [0, 1])
    # rowsCount = sorted(list({row: count for row, count in rows.items() if count < 3}.keys()))

    # dataList = list(chain.from_iterable(expandData(data) for data in xmlDataList if data['row'] not in [0, 1] + rowsCount))
    
    # mergeDataRanges = createRange(dataList)
    # for key in mergeDataRanges:
    #     for row in mergeDataRanges[key]:
    #         tempData = copy.deepcopy(ast.literal_eval(key))
    #         tempData['row'] = row
    #         dataList.append(tempData)

    # result = []
    # sortedDataList = sorted(dataList, key=lambda x: (x['row'], x['col']))
    # for row in list(set([row['row'] for row in sortedDataList])):
    #     customDict = CustomDefaultDict()
    #     rowsGroupList = [data for data in sortedDataList if data['row'] == row]
    #     for datum, column in zip(rowsGroupList, columns):
    #         customDict.append(column['text'], datum['text'])
    #     result.append(dict(customDict))

    

# for i in filteredData:
#     print([j for j in i if j['text']])

# test2([dictData for dictData in filteredData[0] if dictData['text']])
a = [dictData for dictData in filteredData[0] if dictData['text']]
b = [x for x in a]

print(test2(b))

Counter({2: 10, 6: 10, 11: 10, 13: 10, 16: 10, 19: 10, 22: 10, 3: 9, 7: 9, 9: 9, 14: 9, 17: 9, 20: 9, 23: 9, 4: 4, 8: 4, 5: 2, 10: 2, 12: 2, 18: 2, 21: 2, 24: 2, 15: 1})
None
